In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install pytesseract

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:
import urllib3
import requests
import json
import urllib.request
import re
import time
import logging
import pandas
from collections import OrderedDict
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import sys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import math
import pytesseract

In [3]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver', options=chrome_options)

In [4]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/pytesseract'

In [5]:
usr = ""
pwd = ""
 
url1 = "https://www.facebook.com/login/device-based/regular/login/?login_attempt=1&lwv=100"



def date_str(datetime):
 return str(datetime)[:10].replace('-','')

def get_comments(url, no_comments):
    driver = webdriver.Chrome(options=options,executable_path=r'chromedriver')
    
    time.sleep(5)
    driver.get(url1)

    email = driver.find_element_by_xpath("//input[@id='email' or @name='email']")
    email.send_keys(usr)
    password = driver.find_element_by_xpath("//input[@id='pass']")
    password.send_keys(pwd)
    button = driver.find_element_by_xpath("//button[@id='loginbutton']")
    button.click()

    time.sleep(5)

    driver.get(url)
    
    if(no_comments>30):
        for i in range(math.ceil(no_comments/30)-1):
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, '_108_'))).click()

    cmt_div = driver.find_elements_by_class_name('_2a_i')

    comments = []
    for i in cmt_div:
        time.sleep(3)
        try:
            urlt = i.find_element_by_css_selector("[data-sigil='ajaxify']").get_attribute('href')
            driver.execute_script("window.open('', 'new_window')")
            driver.switch_to.window(driver.window_handles[1])
            driver.get(urlt)
            time.sleep(3)
            z = driver.find_elements_by_css_selector("[data-sigil='comment-body']")
            p_comment = z[0].text
            replies = []
            for k in range(1,(len(z))):
                replies.append(z[k].text)

            d = {
                'comment' : p_comment,
                'replies' : replies
            }
            comments.append(d)
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(3)
        except:
            try:
                z = i.find_element_by_class_name('_14v5')
                continue
            except:
                d = {
                    'comment':i.find_element_by_css_selector("[data-sigil='comment-body']").text
                }
                comments.append(d) 
    driver.quit()
    return comments

posts = []
def post_check(post):
    if str(post['time'])[:4]=='2021' or post['post_text'] == None:
        pass
    elif str(post['time'])[:4]=='2020' and re.search(r'#?Amul Topical ?: ?', post['post_text']):
        print(post['post_text'])
        return posts.append(post)

def isPost(post):
    if re.search(r'#?Amul Topical ?: ?', post['text']):
        return True
    else:
        return False
    
def get_data(post_id, no_comments):
    url = 'https://m.facebook.com/amul.coop/posts/a.'+post_id+'/'+post_id
    driver.get(url)
    
    time.sleep(2)
    for i in range(math.ceil(no_comments/30)):
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, '_108_'))).click()
            time.sleep(3)
        except:
            continue

    cmt_div = driver.find_elements_by_class_name('_2a_i')
    
    shares = driver.find_element_by_css_selector("[data-sigil='feed-ufi-sharers']").text.split()[0]
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[1])
    driver.get('https://m.facebook.com/ufi/reaction/profile/browser/?ft_ent_identifier='+post_id)
    time.sleep(2)
    x = driver.find_elements_by_class_name('_10tn')
    r = []
    for i in range(3,len(x)):
        r.append(x[i].find_element_by_tag_name('span').get_attribute('aria-label'))    
         
    reactions = {}
    for i in r:
        reactions[i.split()[-1]]=int(value_to_float(i.split()[0]))
    reactions['shares'] = int(re.sub(',','',shares))
    
    driver.switch_to.window(driver.window_handles[0])
    print('getting comment')
    comments = []
    for i in cmt_div:
        time.sleep(1)
        try:
            urlt = i.find_element_by_css_selector("[data-sigil='ajaxify']").get_attribute('href')
            driver.switch_to.window(driver.window_handles[1])
            driver.get(urlt)
            time.sleep(3)
            z = driver.find_elements_by_css_selector("[data-sigil='comment-body']")
            p_comment = z[0].text
            replies = []
            for k in range(1,(len(z))):
                replies.append(z[k].text)

            d = {
                'comment' : p_comment,
                'replies' : replies
            }
            comments.append(d)
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(3)
        except:
            d = {
                    'comment':i.find_element_by_css_selector("[data-sigil='comment-body']").text
                }
            comments.append(d)
    
    data = {
        'reactions':reactions,
        'comments':comments
    }
    driver.switch_to.window(driver.window_handles[0])
    return data

def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return x

In [6]:
with open('/content/drive/MyDrive/IIM/metadata.json') as f:
  data = json.load(f)

In [7]:
#driver = webdriver.Chrome(options=options,executable_path=r'chromedriver')

driver.get(url1)

email = driver.find_element_by_xpath("//input[@id='email' or @name='email']")
email.send_keys(usr)
password = driver.find_element_by_xpath("//input[@id='pass']")
password.send_keys(pwd)
button = driver.find_element_by_xpath("//button[@id='loginbutton']")
button.click()
driver.execute_script("window.open('', 'new_window')")
driver.switch_to.window(driver.window_handles[0])
time.sleep(5)

In [8]:
results = []

In [9]:
err = []

In [ ]:
for i in range(501,601):
    print(i)
    try:
        urllib.request.urlretrieve(data[i]['image'], "/content/drive/MyDrive/IIM/images/"+data[i]['time']+".jpg")
        x = get_data(data[i]['post_id'], data[i]['comments'])
        x['reactions']['likes'] = data[i]['likes']
        d = {
            'title': re.sub(r'#?Amul Topical ?: ?', '', data[i]['text']),
            'date':data[i]['time'],
            'fb_interactions':x['reactions'],
            'no_fb_comments':data[i]['comments'],
            'fb_comments':x['comments']
        }
        results.append(d)
        if i%20==0:
            print("writing in file")
            y = json.dumps(results, indent=4)
            with open("/content/drive/MyDrive/IIM/data5.json", "w") as outfile: 
                outfile.write(y)
        time.sleep(5)
    except:
        err.append(i)

501
getting comment
502
getting comment
503
getting comment
504
getting comment
505
getting comment
506
getting comment
507
getting comment
508
getting comment
509
getting comment
510
getting comment
511
getting comment


In [ ]:
print('hello world')

In [ ]:
print('hello world')

In [ ]:
len(results)